In [1]:
import requests
import json
import pandas as pd
import altair as alt

/Users/matthewyau/opt/anaconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# Plan

In [2]:
#GRAPHS WE MIGHT WANT TO IMPLEMENT:

#Gaming preferences of friend group
    #Get common games (value counts of games that most people own)
    #Favorite games (show top 3 most played game)
    #Compare genres between two friends


#Leaderboards
    #Total number of hours played
    #Biggest spender
    #Biggest collector (greatest number of games owned)
    #Biggest collector of unplayed games (greatest number of games owned that do not have any hours played in it)

#Profile analysis
    #Genres played
    #Total number of games owned, etc...


# Code

#Always enter the SteamID I pasted below to search for people

In [3]:
#SteamId = 76561197996661065

#### Get a person's friend list off steam ID 

In [4]:
#Implementation:
#Get friend list based off of Steam ID

def get_friend_id(steamId):
    friendIds = []
    try:
        friendlist = requests.get("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&relationship=friend&steamid=" + str(steamId)).json()

        friendIds.append(str(steamId))
        for friend in friendlist["friendslist"]["friends"]:
            friendIds.append(friend["steamid"])
    except:
        print("Error occurred in extracting friend list")

    return friendIds


#### Get all of your own and friends names and profile pictures 

In [5]:
def get_friend_name(friendIds):
    friendSearchQuery = ""
    for i in friendIds:
        friendSearchQuery += i + ","
    try:
        friendName = requests.get("http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=3D41F12368AF3E305A8233ABFB965CA2&steamids=" + friendSearchQuery[:-1]).json()
        friendNameList = {}
        for friendName in friendName["response"]["players"]:
            friendNameList[friendName["personaname"]] = []
            friendNameList[friendName["personaname"]].append(friendName["steamid"])
            friendNameList[friendName["personaname"]].append(friendName["avatar"])
    except:
        print("Error occurred in extracting friend names")
    return friendNameList
    

#get_friend_name(get_friend_id(76561197996661065))

#### Get all of your own and friends game libraries

In [6]:
def get_game_library(friendNameList):
    for friend in friendNameList:
        try:
            gameLibrary = requests.get("https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&include_appinfo=1&format=json&steamid="
                                       + friendNameList[friend][0]).json()["response"]["games"]
            friendNameList[friend].append(gameLibrary)
        except:
            print("Could not load game library for", friend)
    for friend in list(friendNameList):
        if len(friendNameList[friend]) == 2:
            del(friendNameList[friend])
    return friendNameList

In [7]:
#get_game_library(get_friend_name(get_friend_id(76561197996661065)))

#### Get most of game prices for games (if it exists)

In [8]:
#Implementation:
#Get game prices

def get_game_prices(friendNameList):
    appIdList = []
    for friend in friendNameList:
        for game in friendNameList[friend][2]:
            appIdList.append(game["appid"])
    appString = ""
    for app in appIdList:
        appString = appString + str(app) + ","
    prices = requests.get("https://store.steampowered.com/api/appdetails?appids=" + appString + "&filters=price_overview").json()    
    for friend in friendNameList:
        for game in friendNameList[friend][2]:
            try:
                if prices[str(game["appid"])]["success"]:
                    if prices[str(game["appid"])]["data"] != []:
                        game["price"] = float(prices[str(game["appid"])]["data"]["price_overview"]["final_formatted"][5:])
            except:
                pass
    return friendNameList
            
            

In [9]:
friendNameList = get_game_prices(get_game_library(get_friend_name(get_friend_id(76561197996661065))))

### Constuct DataFrame

In [10]:
#Implementation:
#Convert friendList to pandas dataframe
def friendlist_to_dataframe(friendNameList):
    fullLst = []
    
    for friend in friendNameList:
        lst = []
        lst.append(friend) #Append friend name to list
        lst.append(friendNameList[friend][1]) #Append friend avatar to list

        gameLst = [] #Start appending library of games to list
        for game in friendNameList[friend][2]: 
            gameLst.append(game["name"])   
        lst.append(gameLst) #End appending library of games to list
    
        totalNumGamesOwned = len(gameLst) #Append total number of games owned
        lst.append(totalNumGamesOwned)
    
        
        zeroPlaytimeGames = 0 #Append total number of games owned with zero playtime
        for game in friendNameList[friend][2]:
            if game["playtime_forever"] == 0:
                zeroPlaytimeGames += 1
        lst.append(zeroPlaytimeGames)
        
    
        priceLst = []
        for game in friendNameList[friend][2]:
            try:
                priceLst.append(game["price"])
            except:
                pass
        #lst.append(priceLst) #Append purchase price (full price, not on sale) for all games owned
        
        try:
            lst.append(sum(priceLst)) #Uncomment this line if we want the sum of prices for all games owned instead of individual items
        except:
            pass
    
        playtimeLst = []
        for game in friendNameList[friend][2]:
            playtimeLst.append(game["playtime_forever"])
        #lst.append(playtimeLst) #Appending total playtime for all games owned
        lst.append(sum(playtimeLst))
    

    

                                 
    
        fullLst.append(lst)
    return pd.DataFrame(fullLst, columns = ["Name", "ProfilePicture", "GameList", "TotalNumGames", "ZeroPlaytimeGames", "GamePrice", "Playtime"])
    

In [11]:
steamData = friendlist_to_dataframe(friendNameList)

In [12]:

def chart_plotter(dataframe):
    
    dataframe = dataframe.melt(["Name", "ProfilePicture", "GameList"])

    
    
    dataframe["Text"] = (["Total number of games owned"] * dataframe["variable"].value_counts()[0] 
                         + ["Total amount of games owned with zero playtime"] * dataframe["variable"].value_counts()[0] 
                         + ["Total amount of dollars spent on games ($CAD)"] * dataframe["variable"].value_counts()[0] 
                         + ["Total number of minutes spent playing games"] * dataframe["variable"].value_counts()[0] 
                     )
    return dataframe



In [13]:
plotData = chart_plotter(steamData)
plotData

,Name,ProfilePicture,GameList,variable,value,Text
0,卷毛毛秀丽,https://avatars.akamai.steamstatic.com/2f66582...,"[The Forest, Papers, Please, Cuphead, Dark Dec...",TotalNumGames,30.00,Total number of games owned
1,Tedd2008,https://avatars.akamai.steamstatic.com/f3065ac...,"[Terraria, Grand Theft Auto V, Mini Metro, AST...",TotalNumGames,23.00,Total number of games owned
2,audz,https://avatars.akamai.steamstatic.com/fef49e7...,"[Portal 2, Terraria, Starbound, Starbound - Un...",TotalNumGames,19.00,Total number of games owned
3,马铃鼠鼠,https://avatars.akamai.steamstatic.com/302685d...,"[Counter-Strike, Counter-Strike: Condition Zer...",TotalNumGames,63.00,Total number of games owned
4,All About That Money,https://avatars.akamai.steamstatic.com/9f5ee8f...,"[Counter-Strike: Source, Midnight Club II, Bur...",TotalNumGames,49.00,Total number of games owned
5,Stealth,https://avatars.akamai.steamstatic.com/6a054c8...,"[Garry's Mod, Mass Effect (2007), Star Wars: B...",TotalNumGames,76.00,Total number of games owned
6,HenryL,https://avatars.akamai.steamstatic.com/08666eb...,"[Total War: EMPIRE - Definitive Edition, Heart...",TotalNumGames,133.00,Total number of games owned
7,matt,https://avatars.akamai.steamstatic.com/4e8b518...,"[Counter-Strike: Source, Total War: MEDIEVAL I...",TotalNumGames,58.00,Total number of games owned
8,Waul Palker,https://avatars.akamai.steamstatic.com/b0d0557...,"[Battlefield: Bad Company™ 2, Super Monday Nig...",TotalNumGames,40.00,Total number of games owned
9,卷毛毛秀丽,https://avatars.akamai.steamstatic.com/2f66582...,"[The Forest, Papers, Please, Cuphead, Dark Dec...",ZeroPlaytimeGames,13.00,Total amount of games owned with zero playtime


### Dataframe Operations

In [14]:
def compare_chart(dataframe):
    selection_options = alt.binding_select(options = dataframe["variable"].unique().tolist())
    selection = alt.selection_single(name = "Genre", fields = ["variable"], bind = selection_options, init = {"variable": "Playtime"})
    brush = alt.selection_interval(empty = "none")
    
    base = alt.Chart(dataframe).mark_bar().encode(
        x = alt.Y("value", title = [" ", " "]),
        y = alt.Y("Name", title = None, sort = "-x")

    ).transform_filter(selection).properties(height = 800, width = 600, title = "Gaming habits of your friend group")
    
    text = base.mark_text(align = "center", baseline = "top").encode(
        x = alt.value(300),
        y = alt.value(830),
        text = alt.Text("Text"),
        size = alt.value(20)
    )
    
    image = base.mark_image().encode(
        url = "ProfilePicture"
    ).add_selection(selection).add_selection(brush).transform_filter(selection)
    
    c1 = (base + image + text).configure_title(fontSize = 24)
    
    return c1

In [15]:
compare_chart(plotData)

alt.LayerChart(...)

In [16]:
def development_dashboard(dataframe):
    selection_options = alt.binding_select(options = dataframe["variable"].unique().tolist())
    selection = alt.selection_single(name = "Genre", fields = ["variable"], bind = selection_options, init = {"variable": "Playtime"})
    brush = alt.selection_interval(empty = "none")
    
    base = alt.Chart(dataframe).mark_bar().encode(
        x = alt.Y("value", title = [" ", " "]),
        y = alt.Y("Name", title = None, sort = "-x")

    ).transform_filter(selection).properties(height = 800, width = 600, title = "Gaming habits of your friend group")
    
    text = base.mark_text(align = "center", baseline = "top").encode(
        x = alt.value(300),
        y = alt.value(830),
        text = alt.Text("Text"),
        size = alt.value(20)
    )
    
    image = base.mark_image().encode(
        url = "ProfilePicture"
    ).add_selection(selection).add_selection(brush).transform_filter(selection)
    
    c1 = (base + image + text)
    
    c2 = alt.Chart(dataframe).mark_bar(baseline = "middle").encode(
        x = alt.Y("count()"),
        y = alt.X("GameList")
    ).properties(width = 1000, height = 1000).transform_filter(selection).transform_filter(brush)
    
    c3 = (c1 | c2)
    return c3

In [17]:
development_dashboard(plotData).save("filename.html")

### Prototype